# 🏥 Ontario ED Intelligence — Notebook 02
## Module 1: Emergency Department Surge Forecaster

**Goal:** Build a time-series model that predicts ED visit volumes for GTA hospitals,
flagging high-risk surge days up to 7 days in advance.

**Model:** Facebook Prophet (handles seasonality + holidays automatically)

**Hospitals Modelled:**
- Sunnybrook Health Sciences Centre
- Unity Health Toronto (St. Michael's)
- North York General Hospital
- Scarborough Health Network
- Humber River Health
- Trillium Health Partners

## 0️⃣ Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings
import os

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

print('✅ Libraries loaded')
print(f'   pandas  : {pd.__version__}')

## 1️⃣ Generate Realistic Synthetic ED Visit Data
> Based on real Ontario ED patterns:
> - ~200-400 visits/day for large academic hospitals
> - ~100-200 visits/day for community hospitals
> - Monday & Friday spikes, Sunday lows
> - Winter (flu season) surges Dec-Feb
> - Statutory holiday spikes

In [ ]:
np.random.seed(42)

hospitals_config = {
    'Sunnybrook HSC':              {'base': 320, 'noise': 35, 'type': 'Academic'},
    'Unity Health (St. Michaels)': {'base': 290, 'noise': 30, 'type': 'Academic'},
    'North York General':          {'base': 210, 'noise': 25, 'type': 'Community'},
    'Scarborough Health Network':  {'base': 240, 'noise': 28, 'type': 'Community'},
    'Humber River Health':         {'base': 195, 'noise': 22, 'type': 'Community'},
    'Trillium Health Partners':    {'base': 260, 'noise': 30, 'type': 'Community'},
}

# Date range: 3 years of history
dates = pd.date_range(start='2022-01-01', end='2024-12-31', freq='D')

def generate_ed_visits(dates, base, noise):
    n = len(dates)
    visits = np.full(n, float(base))

    # Day-of-week effect
    dow_effect = {0: 1.12, 1: 1.05, 2: 1.00, 3: 1.00, 4: 1.08, 5: 0.92, 6: 0.85}
    for i, d in enumerate(dates):
        visits[i] *= dow_effect[d.dayofweek]

    # Flu season (Dec-Feb boost)
    for i, d in enumerate(dates):
        if d.month in [12, 1, 2]:
            visits[i] *= np.random.uniform(1.15, 1.30)
        elif d.month in [6, 7, 8]:
            visits[i] *= np.random.uniform(0.88, 0.95)

    # Ontario statutory holidays — ED surges
    holidays = [
        '2022-01-01','2022-02-21','2022-04-15','2022-05-23','2022-07-01',
        '2022-08-01','2022-09-05','2022-10-10','2022-12-26',
        '2023-01-01','2023-02-20','2023-04-07','2023-05-22','2023-07-01',
        '2023-08-07','2023-09-04','2023-10-09','2023-12-25','2023-12-26',
        '2024-01-01','2024-02-19','2024-03-29','2024-05-20','2024-07-01',
        '2024-08-05','2024-09-02','2024-10-14','2024-12-25','2024-12-26',
    ]
    holiday_dates = pd.to_datetime(holidays)
    for i, d in enumerate(dates):
        if d in holiday_dates:
            visits[i] *= np.random.uniform(1.20, 1.45)

    # Year-over-year growth (+3% per year — Ontario ED trend)
    for i, d in enumerate(dates):
        years_elapsed = (d.year - 2022)
        visits[i] *= (1.03 ** years_elapsed)

    # Random noise
    visits += np.random.normal(0, noise, n)
    return np.maximum(visits, 50).round().astype(int)

# Build full dataset
all_data = []
for hospital, cfg in hospitals_config.items():
    v = generate_ed_visits(dates, cfg['base'], cfg['noise'])
    for i, d in enumerate(dates):
        all_data.append({
            'date': d,
            'hospital': hospital,
            'type': cfg['type'],
            'ed_visits': v[i],
            'base_capacity': int(cfg['base'] * 1.25)
        })

df = pd.DataFrame(all_data)
df['surge_flag'] = (df['ed_visits'] > df['base_capacity']).astype(int)

print(f'✅ ED visit data generated')
print(f'   Date range  : {df.date.min().date()} → {df.date.max().date()}')
print(f'   Hospitals   : {df.hospital.nunique()}')
print(f'   Total rows  : {len(df):,}')
print(f'   Surge days  : {df.surge_flag.sum()} ({df.surge_flag.mean()*100:.1f}% of all days)')
df.head(10)

## 2️⃣ Exploratory: ED Visit Trends by Hospital

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(20, 14))
axes = axes.flatten()

colours = ['#e41a1c','#377eb8','#4daf4a','#ff7f00','#984ea3','#a65628']

for idx, (hospital, cfg) in enumerate(hospitals_config.items()):
    hdf = df[df['hospital'] == hospital].copy()
    hdf_monthly = hdf.set_index('date')['ed_visits'].resample('ME').mean()

    axes[idx].plot(hdf_monthly.index, hdf_monthly.values,
                   color=colours[idx], linewidth=2)
    axes[idx].fill_between(hdf_monthly.index, hdf_monthly.values,
                            alpha=0.15, color=colours[idx])
    axes[idx].axhline(y=cfg['base'] * 1.25, color='red',
                       linestyle='--', linewidth=1, label='Surge threshold')
    axes[idx].set_title(hospital, fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Avg Daily ED Visits')
    axes[idx].legend(fontsize=9)
    axes[idx].xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    axes[idx].xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    plt.setp(axes[idx].xaxis.get_majorticklabels(), rotation=30)
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_facecolor('#fafafa')

fig.suptitle('GTA Hospital ED Visit Trends (2022-2024)\nMonthly Averages with Surge Thresholds',
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('../reports/ed_trends_by_hospital.png', dpi=150, bbox_inches='tight')
plt.show()
print('✅ Saved → reports/ed_trends_by_hospital.png')

## 3️⃣ Seasonality Analysis: When Do GTA EDs Surge?

In [ ]:
df['month'] = df['date'].dt.month
df['dayofweek'] = df['date'].dt.dayofweek
df['month_name'] = df['date'].dt.strftime('%b')
df['day_name'] = df['date'].dt.strftime('%a')

month_order = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
day_order   = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']

monthly_avg = df.groupby('month_name')['ed_visits'].mean().reindex(month_order)
daily_avg   = df.groupby('day_name')['ed_visits'].mean().reindex(day_order)

fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Monthly pattern
bars1 = axes[0].bar(month_order, monthly_avg.values,
                     color=['#d62728' if m in ['Dec','Jan','Feb'] else '#1f77b4' for m in month_order])
axes[0].set_title('Average ED Visits by Month\n(All GTA Hospitals)',
                   fontsize=13, fontweight='bold')
axes[0].set_ylabel('Average Daily ED Visits')
axes[0].set_xlabel('Month')
axes[0].axhline(y=monthly_avg.mean(), color='grey', linestyle='--', label='Annual average')
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')
for bar, val in zip(bars1, monthly_avg.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                  f'{val:.0f}', ha='center', va='bottom', fontsize=9)

# Day of week pattern
bars2 = axes[1].bar(day_order, daily_avg.values,
                     color=['#d62728' if d in ['Mon','Fri'] else
                            '#aec7e8' if d in ['Sat','Sun'] else '#1f77b4' for d in day_order])
axes[1].set_title('Average ED Visits by Day of Week\n(All GTA Hospitals)',
                   fontsize=13, fontweight='bold')
axes[1].set_ylabel('Average Daily ED Visits')
axes[1].set_xlabel('Day of Week')
axes[1].axhline(y=daily_avg.mean(), color='grey', linestyle='--', label='Weekly average')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')
for bar, val in zip(bars2, daily_avg.values):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                  f'{val:.0f}', ha='center', va='bottom', fontsize=9)

plt.suptitle('Ontario ED Seasonality Patterns — GTA Hospitals',
              fontsize=15, fontweight='bold')
plt.tight_layout()
plt.savefig('../reports/ed_seasonality_patterns.png', dpi=150, bbox_inches='tight')
plt.show()
print('✅ Saved → reports/ed_seasonality_patterns.png')

## 4️⃣ Prophet Forecasting Model
> Training on Sunnybrook HSC — then applying to all hospitals

In [ ]:
# Ontario statutory holidays for Prophet
ontario_holidays = pd.DataFrame({
    'holiday': 'ontario_statutory',
    'ds': pd.to_datetime([
        '2022-01-01','2022-02-21','2022-04-15','2022-05-23',
        '2022-07-01','2022-08-01','2022-09-05','2022-10-10','2022-12-26',
        '2023-01-01','2023-02-20','2023-04-07','2023-05-22',
        '2023-07-01','2023-08-07','2023-09-04','2023-10-09','2023-12-25','2023-12-26',
        '2024-01-01','2024-02-19','2024-03-29','2024-05-20',
        '2024-07-01','2024-08-05','2024-09-02','2024-10-14','2024-12-25','2024-12-26',
        '2025-01-01','2025-02-17','2025-04-18','2025-05-19',
        '2025-07-01','2025-08-04','2025-09-01','2025-10-13','2025-12-25','2025-12-26',
    ]),
    'lower_window': -1,
    'upper_window': 1,
})

print(f'✅ Ontario holidays defined: {len(ontario_holidays)} dates')

def train_prophet(hospital_name, df, holidays):
    hdf = df[df['hospital'] == hospital_name][['date', 'ed_visits']].copy()
    hdf.columns = ['ds', 'y']

    model = Prophet(
        holidays=holidays,
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        seasonality_mode='multiplicative',
        changepoint_prior_scale=0.05,
        holidays_prior_scale=10.0,
        interval_width=0.90
    )
    model.fit(hdf)
    return model, hdf

# Train on Sunnybrook first
print('Training Prophet model on Sunnybrook HSC...')
model_sunny, hdf_sunny = train_prophet('Sunnybrook HSC', df, ontario_holidays)
print('✅ Model trained!')

## 5️⃣ Generate 30-Day Forecast

In [ ]:
# Forecast 30 days ahead
future = model_sunny.make_future_dataframe(periods=30)
forecast = model_sunny.predict(future)

# Surge threshold for Sunnybrook
surge_threshold = int(hospitals_config['Sunnybrook HSC']['base'] * 1.25)

# Plot forecast
fig, ax = plt.subplots(figsize=(18, 7))

# Historical data (last 90 days)
hist_90 = hdf_sunny.tail(90)
ax.scatter(hist_90['ds'], hist_90['y'],
           color='#1f77b4', s=15, alpha=0.6, label='Actual visits', zorder=3)

# Forecast
forecast_only = forecast[forecast['ds'] > hdf_sunny['ds'].max()]
ax.plot(forecast_only['ds'], forecast_only['yhat'],
        color='#ff7f0e', linewidth=2.5, label='Forecast', zorder=4)
ax.fill_between(forecast_only['ds'],
                 forecast_only['yhat_lower'],
                 forecast_only['yhat_upper'],
                 alpha=0.25, color='#ff7f0e', label='90% confidence interval')

# Surge threshold line
ax.axhline(y=surge_threshold, color='red', linestyle='--',
            linewidth=2, label=f'Surge threshold ({surge_threshold} visits/day)')

# Highlight surge days in forecast
surge_days = forecast_only[forecast_only['yhat'] > surge_threshold]
if len(surge_days) > 0:
    ax.scatter(surge_days['ds'], surge_days['yhat'],
               color='red', s=80, zorder=5, label=f'⚠️ Predicted surge days ({len(surge_days)})')

ax.axvline(x=hdf_sunny['ds'].max(), color='grey',
            linestyle=':', linewidth=1.5, label='Forecast start')

ax.set_title('Sunnybrook Health Sciences Centre\nED Visit Forecast — Next 30 Days',
              fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Daily ED Visits')
ax.legend(loc='upper left', fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_facecolor('#fafafa')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%d %b'))
ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=1))
plt.setp(ax.xaxis.get_majorticklabels(), rotation=30)

plt.tight_layout()
plt.savefig('../reports/sunnybrook_forecast.png', dpi=150, bbox_inches='tight')
plt.show()
print(f'✅ Saved → reports/sunnybrook_forecast.png')
print(f'   Predicted surge days: {len(surge_days)}')

## 6️⃣ Train All 6 Hospitals & Build Surge Dashboard

In [ ]:
print('Training Prophet models for all 6 hospitals...')
print('(This takes ~60 seconds)\n')

all_forecasts = {}

for hospital in hospitals_config.keys():
    model, hdf = train_prophet(hospital, df, ontario_holidays)
    future = model.make_future_dataframe(periods=30)
    forecast = model.predict(future)
    forecast_30 = forecast[forecast['ds'] > hdf['ds'].max()].copy()
    forecast_30['hospital'] = hospital
    forecast_30['surge_threshold'] = int(hospitals_config[hospital]['base'] * 1.25)
    forecast_30['surge_flag'] = forecast_30['yhat'] > forecast_30['surge_threshold']
    all_forecasts[hospital] = forecast_30
    surge_count = forecast_30['surge_flag'].sum()
    print(f'  ✅ {hospital:<40} → {surge_count} surge days predicted')

combined = pd.concat(all_forecasts.values(), ignore_index=True)
print(f'\n✅ All models trained!')
print(f'   Total surge days predicted across all hospitals: {combined["surge_flag"].sum()}')

## 7️⃣ 30-Day Surge Risk Dashboard (All Hospitals)

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(20, 16))
axes = axes.flatten()
colours = ['#e41a1c','#377eb8','#4daf4a','#ff7f00','#984ea3','#a65628']

for idx, (hospital, cfg) in enumerate(hospitals_config.items()):
    fc = all_forecasts[hospital]
    threshold = int(cfg['base'] * 1.25)

    axes[idx].plot(fc['ds'], fc['yhat'],
                   color=colours[idx], linewidth=2, label='Forecast')
    axes[idx].fill_between(fc['ds'], fc['yhat_lower'], fc['yhat_upper'],
                            alpha=0.2, color=colours[idx], label='90% CI')
    axes[idx].axhline(y=threshold, color='red', linestyle='--',
                       linewidth=1.5, label=f'Surge threshold')

    surge_fc = fc[fc['surge_flag']]
    if len(surge_fc) > 0:
        axes[idx].scatter(surge_fc['ds'], surge_fc['yhat'],
                           color='red', s=60, zorder=5)

    surge_count = fc['surge_flag'].sum()
    risk_level = '🔴 HIGH' if surge_count > 10 else '🟡 MEDIUM' if surge_count > 4 else '🟢 LOW'

    axes[idx].set_title(f'{hospital}\nRisk: {risk_level} | Surge days: {surge_count}/30',
                         fontsize=11, fontweight='bold')
    axes[idx].set_ylabel('Predicted ED Visits')
    axes[idx].legend(fontsize=8)
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_facecolor('#fafafa')
    axes[idx].xaxis.set_major_formatter(mdates.DateFormatter('%d %b'))
    axes[idx].xaxis.set_major_locator(mdates.WeekdayLocator(interval=2))
    plt.setp(axes[idx].xaxis.get_majorticklabels(), rotation=30)

fig.suptitle('Ontario ED Intelligence Platform\n30-Day Surge Risk Forecast — GTA Hospitals',
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('../reports/gta_surge_dashboard.png', dpi=150, bbox_inches='tight')
plt.show()
print('✅ Saved → reports/gta_surge_dashboard.png')

## 8️⃣ Summary — Surge Risk Table

In [ ]:
summary_rows = []
for hospital, fc in all_forecasts.items():
    surge_days = fc[fc['surge_flag']]
    surge_count = len(surge_days)
    risk = 'HIGH' if surge_count > 10 else 'MEDIUM' if surge_count > 4 else 'LOW'
    summary_rows.append({
        'Hospital': hospital,
        'Type': hospitals_config[hospital]['type'],
        'Avg Forecast (visits/day)': fc['yhat'].mean().round(0).astype(int),
        'Surge Threshold': fc['surge_threshold'].iloc[0],
        'Surge Days (next 30)': surge_count,
        'Risk Level': risk,
        'First Surge Date': surge_days['ds'].min().strftime('%Y-%m-%d') if surge_count > 0 else 'None'
    })

summary_df = pd.DataFrame(summary_rows).sort_values('Surge Days (next 30)', ascending=False)

print('=' * 75)
print('  ONTARIO ED INTELLIGENCE — 30-DAY SURGE RISK SUMMARY')
print('=' * 75)
print(summary_df.to_string(index=False))
print('=' * 75)

summary_df.to_csv('../data/processed/surge_risk_summary.csv', index=False)
print('\n✅ Saved → data/processed/surge_risk_summary.csv')
print('🚀 Next → Module 2: ALC Bed Block Analyzer')